In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import sys
# import os
sys.path.append('/Users/isaacwatson/Documents/MSc CSML/COMP0124/research-project/MultiAgentTrading/')

In [9]:
# pull data from yahoo finance

import yfinance as yf
import pandas as pd
import numpy as np

# def get_data(tickers: list, start_date: str, end_date: str) -> pd.DataFrame:
#     """
#     Pull data from Yahoo Finance.
#     Download individually and merge the data by the date.

#     Args:
#         tickers: list
#             The tickers of the assets.
#         start_date: str
#             The start date of the data.
#         end_date: str
#             The end date of the data.

#     Returns:
#         data: pd.DataFrame
#             The data of the assets.
#     """
#     data = pd.DataFrame()
#     for ticker in tickers:
#         df = yf.download(ticker, start=start_date, end=end_date)
#         df['ticker'] = ticker
#         data = pd.concat([data, df])
    
#     # group by date and ticker
#     data = data.reset_index()
#     data = data.groupby('Date').apply(lambda x: x.set_index('ticker'))
#     data = data.drop('Date', axis=1)
#     return data

# def get_dj_tickers():
#     """
#     Get the tickers of the Dow Jones 30 companies from Wikipedia.
#     """
#     url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'
#     html = pd.read_html(url, header=0)
#     dj = html[1]
#     tickers = dj.Symbol.to_list()
#     return tickers

# ticker_list = get_dj_tickers()
ticker = "000001.SS"

start_date = '1990-11-05'
end_date = '2021-05-31'

stock = yf.Ticker(ticker)
price = stock.history(period="max")

# data = get_data(ticker_list, start_date, end_date)
df = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [10]:
df.to_csv(f'../datasets/{ticker}-daily.csv')

In [11]:
# compute technical indicators: MACD, RSI, CCI, and ADX
from ta.trend import macd
from ta.momentum import rsi
from ta.trend import cci
from ta.trend import adx

def compute_technical_indicators(data: pd.DataFrame) -> pd.DataFrame:
    """
    Compute technical indicators: MACD, RSI, CCI, and ADX.

    Args:
        data: pd.DataFrame
            The data of the assets.

    Returns:
        data: pd.DataFrame
            The data of the assets with technical indicators.
    """
    data['macd'] = macd(data['Close'])
    data['rsi'] = rsi(data['Close'])
    data['cci'] = cci(data['High'], data['Low'], data['Close'])
    data['adx'] = adx(data['High'], data['Low'], data['Close'])
    return data

df = compute_technical_indicators(df)

In [12]:
df.dropna(inplace=True)
df.rename(columns={"macd": "MACD", "rsi": "RSI", "cci": "CCI", "adx": "ADX"}, inplace=True)
df

,Open,High,Low,Close,Adj Close,Volume,MACD,RSI,CCI,ADX
Date,,,,,,,,,,
1997-08-07,1167.927002,1199.239990,1161.631958,1199.239990,1199.239990,0,2.550960,54.645586,22.845270,0.000000
1997-08-08,1205.952026,1216.561035,1201.358032,1206.052979,1206.052979,0,4.060648,56.272215,103.109085,0.000000
1997-08-11,1206.456055,1206.759033,1182.447021,1184.083008,1184.083008,0,3.444587,50.039740,21.026857,8.973011
1997-08-12,1177.260986,1183.436035,1145.748047,1146.808960,1146.808960,0,-0.050765,41.617877,-129.466958,10.228693
1997-08-13,1139.245972,1153.494995,1118.202026,1120.126953,1120.126953,0,-4.917189,36.838301,-205.484591,11.999077
...,...,...,...,...,...,...,...,...,...,...
2021-05-24,3486.270020,3498.300049,3469.870117,3497.280029,3497.280029,289500,13.635196,55.048920,60.346038,12.810541
2021-05-25,3502.540039,3584.580078,3502.439941,3581.340088,3581.340088,341200,20.298158,66.047907,188.949292,14.277405
2021-05-26,3586.840088,3603.489990,3585.370117,3593.360107,3593.360107,342000,26.245971,67.280773,208.030679,15.944108


In [13]:
df.to_csv(f'../datasets/{ticker}-daily.csv')

In [16]:
train_data = df.loc[:'2019-12-01']
test_data = df.loc['2019-12-01':'2021-05-31']

In [17]:
train_data.to_csv(f'../datasets/{ticker}-train.csv')
test_data.to_csv(f'../datasets/{ticker}-test.csv')